## 使用机器学习抽取信号特征的时序分类方法

### 1.特征抽取

### 2.特征选择

### 3.model stacking


In [2]:
import pandas as pd
# from processing_utils import *
import warnings
from sklearn.metrics import classification_report
warnings.filterwarnings('ignore')
import numpy as np
np.seterr(divide='ignore',invalid='ignore')
import _pickle as pkl
def load_file_with_p(file_path):
    '''载入,p二进制文件'''
    return pkl.load(open(file_path,'rb'))

def save_file_with_p(save_df,file_path):
    '''载入,p二进制文件'''
    pkl.dump(save_df,open(file_path,'wb'))

def remove_highly_correlated_columns(df, threshold):
    # 计算相关系数矩阵
    corr_matrix = df.corr().abs()

    # 创建一个布尔值的DataFrame，表示相关系数是否大于阈值
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool_))

    # 获取要删除的列名
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    print(to_drop)
    # 删除相关系数较高的列
    df = df.drop(to_drop, axis=1)

    return df,to_drop,upper

## 数据处理


In [2]:
from tsfresh import extract_features
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute

train_X = np.load('训练集/train_x.npy') 
train_y = np.load('训练集/train_y.npy') 


test_X = np.load('测试集A/test_x_A.npy')
df_test_y = pd.read_csv('测试集A/submit_example_A.csv')

df = pd.DataFrame(columns=['id','time','f1','f2'])

for index,i in enumerate(test_X):
    tdf = pd.DataFrame(i).T
    tdf.columns = ['f1','f2']
    tdf['id'] = index
    tdf['time'] = [j for j in range(len(tdf))]
    
    tdf = tdf[['id','time','f1','f2']]
    df = pd.concat([df,tdf])


df.to_csv('/home/gsl/workspace/jd/data/test.csv',index=False)


df1 = pd.read_csv('/train_1.csv')
df2 = pd.read_csv('/train_2.csv')
df_test = pd.read_csv(/test.csv')
df2['id'] = df2['id'] + 15000
df = pd.concat([df1,df2])

extracted_features = extract_features(df, column_id="id", column_sort="time")
extracted_features_test = extract_features(df_test, column_id="id", column_sort="time")

impute(extracted_features)
features_filtered = select_features(extracted_features, train_y)
impute(extracted_features_test)
features_filtered_test = extracted_features_test[features_filtered.columns.tolist()]

save_file_with_p(features_filtered,'/train.p')
save_file_with_p(features_filtered_test,'/test.p')

# 加载数据


In [96]:
features_filtered = load_file_with_p(/train.p')
features_filtered_test = load_file_with_p('/test.p')
y_train = np.load('train_y.npy') 

In [97]:
pd.Series(y_train).value_counts()

0    29808
2     4520
1     3221
dtype: int64

提升方式：

特征优化
模型融合


## 1.特征优化


In [98]:
# 1.去除冗余特征

# x_train,to_drop,upper = remove_highly_correlated_columns(features_filtered,threshold=0.7)
# x_test = features_filtered_test[x_train.columns.tolist()]

x_train = features_filtered
x_test = features_filtered_test

# 2.修改特征列名
source_cols = x_train.columns.tolist()
x_train.columns = [f'fea_{i}' for i in range(len(source_cols))]
x_test.columns = [f'fea_{i}' for i in range(len(source_cols))]

## 2.模型融合


In [99]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score  
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

def cv_model(clf, train_x, train_y, test_x, clf_name, seed = 2023):
    folds = 5
    kf = KFold(n_splits=folds, shuffle=True, random_state=seed)
    num_class = 3 
    oof = np.zeros((train_x.shape[0], num_class))  # 适应三分类
    test_predict = np.zeros((test_x.shape[0], num_class))  # 适应三分类
    cv_scores = []
    for i, (train_index, valid_index) in enumerate(kf.split(train_x, train_y)):
        print('************************************ {} ************************************'.format(str(i+1)))
        trn_x, trn_y, val_x, val_y = train_x.iloc[train_index], train_y[train_index], train_x.iloc[valid_index], train_y[valid_index]
        
        if clf_name == "lgb":
            train_matrix = clf.Dataset(trn_x, label=trn_y)
            valid_matrix = clf.Dataset(val_x, label=val_y)
            params = {
                'boosting_type': 'gbdt',
                'objective': 'multiclass',
                'min_child_weight': 6,
                'num_leaves': 2 ** 6,
                'lambda_l2': 10,
                'feature_fraction': 0.8,
                'bagging_fraction': 0.8,
                'bagging_freq': 4,
                'learning_rate': 0.35,
                'seed': 2024,
                'nthread' : 16,
                'verbose' : -1,
                'num_class':num_class
            }
            model = clf.train(params, train_matrix, 2000, valid_sets=[train_matrix, valid_matrix],
                              categorical_feature=[], verbose_eval=1000, early_stopping_rounds=100)
            val_pred = model.predict(val_x, num_iteration=model.best_iteration)
            test_pred = model.predict(test_x, num_iteration=model.best_iteration)
        
        if clf_name == "xgb":

            xgbc = XGBClassifier(reg_lambda = 9.0,
                            reg_alpha = 2.93,
                            gamma=5,
                            max_depth=15,
                            colsample_bytree = 0.6,
                            subsample = 0.7,
                            learning_rate = 0.35,
                            n_estimators = 60,
                            min_child_weight = 7,
                            objective='multi:softmax',
                            n_jobs=12)
            
            # model = clf.train(xgbc, train_matrix, num_boost_round=2000, evals=watchlist, verbose_eval=1000, early_stopping_rounds=100)
            xgbc.fit(trn_x,trn_y)
            val_pred  = xgbc.predict_proba(val_x)
            test_pred = xgbc.predict_proba(test_x)

        if clf_name == "cat":
            params = {'learning_rate': 0.35, 'depth': 5, 'bootstrap_type':'Bernoulli','random_seed':2024,
                      'od_type': 'Iter', 'od_wait': 100, 'random_seed': 11, 'allow_writing_files': False,
                      'loss_function':'MultiClass'}
            
            model = clf(iterations=2000, **params)
            model.fit(trn_x, trn_y, eval_set=(val_x, val_y),
                      metric_period=1000,
                      use_best_model=True, 
                      cat_features=[],
                      verbose=1)
            
            val_pred  = model.predict_proba(val_x)
            test_pred = model.predict_proba(test_x)
        
        oof[valid_index] = val_pred
        test_predict += test_pred / kf.n_splits
        
        F1_score = accuracy_score(val_y, np.argmax(val_pred, axis=1))  # 适应三分类
        cv_scores.append(F1_score)
        print(cv_scores)
        
    return oof, test_predict

# 参考demo,具体对照baseline实践部分调用cv_model函数
# 选择lightgbm模型
lgb_oof, lgb_test = cv_model(lgb, x_train, y_train, x_test, 'lgb')
# 选择xgboost模型
xgb_oof, xgb_test = cv_model(xgb, x_train, y_train, x_test, 'xgb')
# 选择catboost模型
cat_oof, cat_test = cv_model(CatBoostClassifier, x_train, y_train, x_test, 'cat')

# # 进行取平均融合
# final_test = (lgb_test + xgb_test + cat_test) / 3

************************************ 1 ************************************
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[10]	training's multi_logloss: 0.239915	valid_1's multi_logloss: 0.375426
[0.8605858854860187]
************************************ 2 ************************************
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[9]	training's multi_logloss: 0.254084	valid_1's multi_logloss: 0.370691
[0.8605858854860187, 0.8628495339547271]
************************************ 3 ************************************
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[12]	training's multi_logloss: 0.22103	valid_1's multi_logloss: 0.385156
[0.8605858854860187, 0.8628495339547271, 0.8541944074567244]
************************************ 4 ************************************
Training until validation scores don't improve for 100 rounds


0:	learn: 0.7307551	test: 0.7328018	best: 0.7328018 (0)	total: 104ms	remaining: 3m 26s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.3721735634
bestIteration = 116

Shrink model to first 117 iterations.
[0.8607190412782956]
************************************ 2 ************************************


0:	learn: 0.7306870	test: 0.7291676	best: 0.7291676 (0)	total: 97.6ms	remaining: 3m 15s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.3656341428
bestIteration = 127

Shrink model to first 128 iterations.
[0.8607190412782956, 0.8607190412782956]
************************************ 3 ************************************


0:	learn: 0.7339951	test: 0.7375335	best: 0.7375335 (0)	total: 99.4ms	remaining: 3m 18s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.3741820356
bestIteration = 126

Shrink model to first 127 iterations.
[0.8607190412782956, 0.8607190412782956, 0.859121171770972]
************************************ 4 ************************************


0:	learn: 0.7328815	test: 0.7319509	best: 0.7319509 (0)	total: 101ms	remaining: 3m 21s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.3597586346
bestIteration = 78

Shrink model to first 79 iterations.
[0.8607190412782956, 0.8607190412782956, 0.859121171770972, 0.8617842876165113]
************************************ 5 ************************************


0:	learn: 0.7306804	test: 0.7296355	best: 0.7296355 (0)	total: 97.5ms	remaining: 3m 14s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.3613000029
bestIteration = 108

Shrink model to first 109 iterations.
[0.8607190412782956, 0.8607190412782956, 0.859121171770972, 0.8617842876165113, 0.8634971367692102]


In [100]:
# 获取预测标签
lgb_df = pd.DataFrame(np.argmax(lgb_oof,axis=1),columns=['lgb'])
xgb_df = pd.DataFrame(np.argmax(xgb_oof,axis=1),columns=['xgb'])
cat_df = pd.DataFrame(np.argmax(cat_oof,axis=1),columns=['cat'])

lgb_test_df = pd.DataFrame(np.argmax(lgb_test,axis=1),columns=['lgb'])
xgb_test_df = pd.DataFrame(np.argmax(xgb_test,axis=1),columns=['xgb'])
cat_test_df = pd.DataFrame(np.argmax(cat_test,axis=1),columns=['cat'])

In [101]:
print(lgb_test_df['lgb'].value_counts())
print(xgb_test_df['xgb'].value_counts())
cat_test_df['cat'].value_counts()

0    791
1    232
2    132
Name: lgb, dtype: int64
0    779
1    245
2    131
Name: xgb, dtype: int64


0    790
1    238
2    127
Name: cat, dtype: int64

In [102]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.ensemble  import RandomForestClassifier


def stack_model(oof_1, oof_2, oof_3, predictions_1, predictions_2, predictions_3, y):
    '''
    输入的oof_1, oof_2, oof_3可以对应lgb_oof，xgb_oof，cat_oof
    predictions_1, predictions_2, predictions_3对应lgb_test，xgb_test，cat_test
    '''
    train_stack = pd.concat([oof_1, oof_2, oof_3], axis=1)
    test_stack = pd.concat([predictions_1, predictions_2, predictions_3], axis=1)
    
    oof = np.zeros((train_stack.shape[0],))
    predictions = np.zeros((test_stack.shape[0],))
    scores = []
    
    from sklearn.model_selection import RepeatedKFold
    folds = RepeatedKFold(n_splits=5, n_repeats=2, random_state=2021)
    
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_stack, train_stack)): 
        print("fold n°{}".format(fold_+1))
        trn_data, trn_y = train_stack.loc[trn_idx], y[trn_idx]
        val_data, val_y = train_stack.loc[val_idx], y[val_idx]
        
        clf = RandomForestClassifier(n_estimators=100)

        clf.fit(trn_data, trn_y)

        oof[val_idx] = clf.predict(val_data)
        predictions += clf.predict(test_stack) / (5 * 2)

        score_single = accuracy_score(val_y, oof[val_idx])
        scores.append(score_single)
        print(f'{fold_+1}/{5}', score_single)
    print('mean: ',np.mean(scores))
   
    return oof, predictions


oof, predictions = stack_model(lgb_df,xgb_df,cat_df,lgb_test_df,xgb_test_df,cat_test_df,y_train)

fold n°1
1/5 0.8624500665778961
fold n°2
2/5 0.8588548601864181
fold n°3
3/5 0.8668442077230359
fold n°4
4/5 0.8693741677762983
fold n°5
5/5 0.8653615661206552
fold n°6
6/5 0.8653794940079893
fold n°7
7/5 0.8655126498002663
fold n°8
8/5 0.863115845539281
fold n°9
9/5 0.8617842876165113
fold n°10
10/5 0.8630976161939007
mean:  0.8641774761542251


In [105]:
res_df = pd.DataFrame(predictions,columns=['label'])
res_df['id'] = res_df.index
res_df = res_df[['id','label']]
res_df['label'] = res_df['label'].round().astype(int)
res_df.to_csv('submit.csv',index=False)

In [104]:
res_df['label'].value_counts()

0    789
1    256
2    110
Name: label, dtype: int64